# Description

TODO

# Modules loading

In [1]:
%env CM_N_JOBS=3
%env NUMBA_NUM_THREADS=3
%env MKL_NUM_THREADS=3
%env OPEN_BLAS_NUM_THREADS=3
%env NUMEXPR_NUM_THREADS=3
%env OMP_NUM_THREADS=3

env: CM_N_JOBS=3
env: NUMBA_NUM_THREADS=3
env: MKL_NUM_THREADS=3
env: OPEN_BLAS_NUM_THREADS=3
env: NUMEXPR_NUM_THREADS=3
env: OMP_NUM_THREADS=3


In [2]:
from time import time

import numpy as np
import pandas as pd
from scipy.stats import pearsonr, spearmanr

from clustermatch import conf
from clustermatch.coef import cm
from clustermatch.methods import mic

# Settings

In [3]:
OUTPUT_FILENAME = "time_test.pkl"

In [4]:
DATA_SIZES = [
    100,
    500,
    1000,
    5000,
    10000,
    50000,
    100000,
    1000000,
]

N_REPS = 10

In [5]:
np.random.seed(0)

# Paths

In [6]:
OUTPUT_DIR = conf.RESULTS_DIR / "time_test"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
display(OUTPUT_DIR)

PosixPath('/opt/data/results/time_test')

# Functions

In [7]:
# append to previous run
time_results = pd.read_pickle(OUTPUT_DIR / OUTPUT_FILENAME)

In [8]:
time_results.shape

(4220, 4)

In [9]:
def run_method(func, method_name, size):
    n_reps = N_REPS
    if size < 500:
        n_reps = 1000

    for r in range(n_reps):
        d1 = np.random.rand(size)
        d2 = np.random.rand(size)

        start_time = time()
        sim = func(d1, d2)
        end_time = time()
        met_time = end_time - start_time

        idx = time_results.shape[0]
        time_results.loc[idx] = [d1.shape[0], method_name, met_time, sim]

# Run

In [10]:
# initialize methods
cm(np.random.rand(100), np.random.rand(100))

0.02047038824894002

In [11]:
for s in DATA_SIZES:
    print(f"Size: {s}")

    print(f"  p")
    run_method(lambda x, y: pearsonr(x, y)[0], "p-3", s)

    print(f"  s")
    run_method(lambda x, y: spearmanr(x, y)[0], "s-3", s)

    print(f"  cm")
    run_method(lambda x, y: cm(x, y), "cm-3", s)

    if s <= 10000:
        print(f"  mic")
        run_method(lambda x, y: mic(x, y), "mic-3", s)

    if s <= 50000:
        print(f"  mic_e")
        run_method(lambda x, y: mic(x, y, estimator="mic_e"), "mic_e-3", s)

    print("Saving to pickle")
    time_results.to_pickle(OUTPUT_DIR / OUTPUT_FILENAME)

Size: 100
  p
  s
  cm
  mic
Saving to pickle
Size: 500
  p
  s
  cm
  mic
Saving to pickle
Size: 1000
  p
  s
  cm
  mic
Saving to pickle
Size: 5000
  p
  s
  cm
  mic
Saving to pickle
Size: 10000
  p
  s
  cm
  mic
Saving to pickle
Size: 100000
  p
  s
  cm
Saving to pickle
Size: 1000000
  p
  s
  cm
Saving to pickle


In [12]:
time_results.shape

(8440, 4)